# Practica 3
Nombre: Ulises Abdiel Cabello Cardenas

Para esta practica primero se importa lo que se va a ocupar:
- Torch: que permite trabajar con tensores.
- torch.nn: que tiene las herramientas para las redes neuronales como las funciones de activacion y de perdida
- torch.optim: incluye los optimizadores
- Dataset y dataloader: carga los datos por lotes

In [21]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

## Carga del dataset
Primero creamos el dataset de la tabla dada

In [22]:
class Automata_dta(Dataset):
    def __init__(self):
        self.X = torch.tensor([
            [0, 0, 1],  
            [0, 0, 0],  
            [0, 1, 1],  
            [0, 1, 0],  
            [1, 0, 1],  
            [1, 0, 0],  
            [1, 1, 1],  
            [1, 1, 0],  
        ], dtype=torch.float32)

        self.Y = torch.tensor([
            [0, 0],  
            [0, 1],  
            [0, 0],  
            [1, 0],  
            [1, 0],  
            [1, 1],  
            [1, 0],  
            [0, 0],  
        ], dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]

## Carga de datos y separacion
En esta parte cargamos los datos y lo separamos por lotes, se escoje separar los datos por lotes de 4 y mezclamos los datos

In [23]:
dataset = Automata_dta()
entrenamiento = DataLoader(dataset, batch_size=4, shuffle=True)

## Creacion de la red neuronal
Creamos una red neuronal para le automata, para esto tomamos 3 capas de entrada y 5 ocultas que van a pasar a las de salida que son 2, en este caso se toma la funcion sigmoide que es parecida a la funcion logistica solo que esta empieza en 0, se toma esta funcion de activacion ya que es la solicitada y es muy buena cuando se requieren valores binarios en la salida, aunque tenga inconvenientes como el desvanecimiento de gradiente, ya que es muy buena para valores muy cercanos al cero, pero cuando se aleja la derivada es muy peque;a y cuando se hace el backgroud la red aprendera poco.

Para la funcion forward los datos fluiran en la red neuronal hacia adelante con la funcion sigmoide.


In [24]:
class ANN(nn.Module):
    def __init__(self):
        super(ANN, self).__init__()
        self.fc1 = nn.Linear(3, 5)  # Capa oculta de 5 neuronas
        self.sigmoid = nn.Sigmoid() # Función de activación
        self.fc2 = nn.Linear(5, 2)  # Capa de salida de 2 neuronas

    def forward(self, x):
        x = self.sigmoid(self.fc1(x))  # Capa oculta + activación Sigmoide
        x = self.sigmoid(self.fc2(x))  # Capa de salida + activación Sigmoide
        return x

## Inicializacion
En esta parte del codigo se instancia la red y se definel la funcion de perdida que ayda a determinar que tan bien esta prediciendo la red neuronal, como optimizador de la red se toma el SGD que es el gradiente descendiente.

In [25]:
model = ANN()
criterio = nn.BCELoss()  # Binary Cross Entropy
optimizer = optim.SGD(model.parameters(), lr=0.05)  

## Entrenamos el modelo
Para ello tomamos 10000 iteraciones que se van a hacer al modelo, primero inicializamos la perdida en cero e iteramos sobre los lotes que se partieron, despues reiniciamos los gradientes y en `outputs = model(inputs)` calculamos las predicciones de la red neuronal, despues vemos y se compara los valores de el anterior con los valores reales aqui se ocupa el BCE de perdida, en `loss.backward()` se calculan los grdientes y es la fase donde se modifican los pesos posteriormente se actualizan y se cambia el total de perdida.

Al ultimo solo se muestra la perdida de la ultima iteracion

In [26]:
epochs = 10000  
for epoch in range(epochs):
    total_loss = 0.0
    for inputs, labels in entrenamiento:
        optimizer.zero_grad()  # Reiniciamos gradientes
        outputs = model(inputs)  # Forward propagation
        loss = criterio(outputs, labels)  # Cálculo de pérdida
        loss.backward()  # Backward propagation
        optimizer.step()  # Actualización de pesos
        total_loss += loss.item()

    if (epoch + 1) % 10000 == 0:
        print(f'Época [{epoch+1}/{epochs}], Pérdida: {total_loss/len(entrenamiento):.4f}')

Época [10000/10000], Pérdida: 0.0495


## Evaluacion del modelo

En esta parte se evalua el modelo y se redondean los valores con round

In [27]:
with torch.no_grad():
    # Evaluación del modelo
    test_outputs = model(dataset.X)
    
    #  Convertir valores a 0 o 1 usando `round()`
    predicted = test_outputs.round()
    
    print("\nPredicciones finales:")
    print(predicted)


Predicciones finales:
tensor([[0., 0.],
        [0., 1.],
        [0., 0.],
        [1., 0.],
        [1., 0.],
        [1., 1.],
        [1., 0.],
        [0., 0.]])
